###컴퓨터에게 단어를 이해시키기
- 시소러스(사람이 만든 유의어 사전)
- 통계 기반
- 추론 기반

###시소러스
- 유의어를 한 그룹으로 분류
- 단어들의 관계를 그래프로 표현하여 단어 사이의 연결 정의
- ex) WordNet

- 문제점
    - 시대 변화에 대응 어려움
    - 구성을 위한 인건비 많이 듬
    - 단어의 미묘한 차이 표현 불가

###통계 기반
- 말뭉치(corpus)
    - 자연어 처리를 위해 수집된 텍스트 데이터


In [1]:
#Example
text = 'You say goodbye and I say hello.'
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [2]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [3]:
#단어 -> ID//ID-> 단어 dictionary 만들기
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

print(id_to_word)
print(word_to_id)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}


In [4]:
#단어 목록 -> 단어 id 목록
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5 6]


In [5]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

단어 -> 벡터로 표현
올바른 벡터 표현을 단어에서 구축 가능할까? (RGB를 보고 대충 무슨색인지 알 수있는 느낌)
단어의 의미를 정확하게 파악할 수 있는 벡터 표현 --> 분산표현

분포가설 : 단어의 의미는 주변 단어에 의해 형성된다
-> 단어가 사용된 맥락(context)이 의미를 형성

동시 발생 행렬 -> 각 단어마다 동시에 발생하는 맥락이 있는 단어의 빈도

In [6]:
#동시발생행렬 구하기
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size=len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx-1
            right_idx = idx+1

            if left_idx>=0:
                left_world_id = corpus[left_idx]
                co_matrix[word_id, left_world_id] +=1

            if right_idx<corpus_size:
                right_world_id = corpus[right_idx]
                co_matrix[word_id, right_world_id] += 1
            

    return co_matrix



In [8]:
#벡터간 유사도를 구하기 위한 cosine similarity : 단어 벡터 유사도를 구할 떄 자주 사용
def cos_similarity(x, y, eps=1e-8):
    nx = x/ (np.sqrt(np.sum(x**2))+eps)
    ny = y/ (np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx,ny)

In [9]:
#you단어 벡터와 i 단어벡터의 단어유사도 구해보기
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0,c1))

0.7071067691154799


In [10]:
#유사 단어의 랭킹 표시
#어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('Can\'t find the word %s.'%query)
        return
    
    print('\n[query] '+query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    #cos similarity 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    #cos similarity top 5
    count = 0
    for i in (-1*similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s'%(id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [12]:
#argsort() : 배열의 원소를 오름차순으로 정렬하고 배열의 인덱스를 반환
x = np.array([100, -20, 2])
print(x.argsort())

[1 2 0]
[100 -20   2]


In [13]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


How to improve?
